<a href="https://colab.research.google.com/github/AS2017521upeksha/ML1_Project/blob/main/data_scraping_part_from_imdb_site.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from time import time
from time import sleep
from random import randint
from IPython.core.display import clear_output
from warnings import warn
import pandas as pd


import re

from bs4 import BeautifulSoup

from requests import get

headers = {"Accept-Language": "en-US, en;q=0.5"}

fromyear = int(input("Enter From Year: "))
tillyear = int(input("Till Year: "))


# Redeclaring the lists to store data in
names = []
years = []
imdb_ratings = []
directors = []
actors_1 = []
actors_2 = []
actors_3 = []
actors_4 = []
generes = []
run_time = []
writers = []
producers = []
countries = []
sounds = []
budget = []
pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(fromyear,tillyear)]

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every year in the interval 2000-2017
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('http://www.imdb.com/search/title?release_date=' + year_url + 
        '&sort=num_votes,desc&page=' + page, headers = headers)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 150:
            warn('Number of requests was greater than expected.')  
            break 

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year 
                year = container.h3.find('span', class_ = 'lister-item-year').text
                year = year.replace("(","")
                year = year.replace(")","")
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                #Scrape the director name
                director_name = container.find_all("p")[2].a.text
                directors.append(director_name)

                #scrape the 4 members of cast
                c = container.find_all("p")[2].find_all("a")[1:5]
                cast_1 = c[0].get_text()
                cast_2 = c[1].get_text()
                cast_3 = c[2].get_text()
                cast_4 = c[3].get_text()
                actors_1.append(cast_1)
                actors_2.append(cast_2)
                actors_3.append(cast_3)
                actors_4.append(cast_4)
                #scrape the gneres
                genre = container.find_all("p")[0].find("span",class_="genre").string.split(",")[0].strip()
                generes.append(genre)
                #scrape the run time
                runtime = int(container.find_all("p")[0].find("span",class_="runtime").string.split("min")[0].strip())#split the min from data
                run_time.append(runtime)

                l = "http://www.imdb.com/"+container.h3.a['href'] #go to inside of movie page
            
                response_movie = get(l)

                movie_html = BeautifulSoup(response_movie.text,'html.parser')

                #scrape the the writer name
                writer = movie_html.find_all("div",class_="credit_summary_item")[1].a.get_text().strip()
                writers.append(writer)

                text = movie_html.find_all("div",id="titleDetails")[0].get_text()
                # remove whitespaces
                text = re.sub(r"\s+", "", text)

                if len(re.findall(r"ProductionCo:[A-Z,a-z]+",text)) != 0:  #srape the production name by ProductionCo word because contain class is defferent from one to another
                   productionco = re.findall(r"ProductionCo:[A-Z,a-z]+",text)[0]
                   production = productionco.split(":")[1].split(",")[0]
                else:
                   production = "NA"
                producers.append(production)

                if len(re.findall(r"Country:[A-Z,a-z]+",text)) != 0:  #scrape the country
                   country = re.findall(r"Country:[A-Z,a-z]+",text)[0]
                   country = country.split(":")[1].split(",")[0]
                else:
                   country = "NA"
                countries.append(country )

                if len(re.findall(r"SoundMix:[A-Z,a-z]+",text)) != 0: #scrape the sound mix
                   sound = re.findall(r"SoundMix:[A-Z,a-z]+",text)[0]
                   sound = sound.split(":")[1].split(",")[0]
                else:
                   sound = "NA"
                sounds.append(sound)

                text = text.replace(",","")
                if len(re.findall(r"Budget:\$[0-9]+",text)) != 0: #scrape the budget
                   bt = re.findall(r"Budget:\$[0-9]+",text)[0]
                   bt = float(bt.split(":")[1].split("$")[1])
                else:
                   bt = 0
                
                budget.append(bt)
               

movie_ratings = pd.DataFrame({'movie': names,
                              'year': years,
                              'imdb': imdb_ratings,
                              'director_name' : directors,
                              'actor_1_name' : actors_1,
                              'actor_2_name' : actors_2,
                              'actor_3_name' : actors_3,
                              'actor_4_name' : actors_4,
                              'gener' : generes,
                              'run_time' : run_time,
                              'writer' : writers,
                              'production_house' : producers,
                              'country' : countries,
                              'sounds' : sounds,
                              'budget' : budget})

print(movie_ratings.info())
movie_ratings.head(10)

movie_ratings.to_csv('moviedata.csv', index=False)